In [14]:
import numpy as np
import pandas as pd
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sympy.geometry import Point3D

structures = pd.read_csv('./input/structures.csv')
molecule_names = structures.molecule_name.unique()

# initiate the plotly notebook mode
init_notebook_mode(connected=True)


def plot_molecule(molecule_name, structures_df):
    """Creates a 3D plot of the molecule"""
    
    atomic_radii = dict(C=0.77, F=0.71, H=0.38, N=0.75, O=0.73)  
    cpk_colors = dict(C='black', F='green', H='white', N='blue', O='red')

    molecule = structures_df[structures_df.molecule_name == molecule_name]
    coordinates = molecule[['x', 'y', 'z']].values
    x_coordinates = coordinates[:, 0]
    y_coordinates = coordinates[:, 1]
    z_coordinates = coordinates[:, 2]
    elements = molecule.atom.tolist()
    radii = [atomic_radii[element] for element in elements]
    
    def get_bonds():
        """Generates a set of bonds from atomic cartesian coordinates"""
        ids = np.arange(coordinates.shape[0])
        bonds = dict()
        coordinates_compare, radii_compare, ids_compare = coordinates, radii, ids
        
        for _ in range(len(ids)):
            coordinates_compare = np.roll(coordinates_compare, -1, axis=0)
            radii_compare = np.roll(radii_compare, -1, axis=0)
            ids_compare = np.roll(ids_compare, -1, axis=0)
            distances = np.linalg.norm(coordinates - coordinates_compare, axis=1)
            bond_distances = (radii + radii_compare) * 1.3
            mask = np.logical_and(distances > 0.1, distances <  bond_distances)
            distances = distances.round(2)
            new_bonds = {frozenset([i, j]): dist for i, j, dist in zip(ids[mask], ids_compare[mask], distances[mask])}
            bonds.update(new_bonds)
        return bonds            
            
    def atom_trace():
        """Creates an atom trace for the plot"""
        colors = [cpk_colors[element] for element in elements]
        markers = dict(color=colors, line=dict(color='lightgray', width=2), size=7, symbol='circle', opacity=0.8)
        trace = go.Scatter3d(x=x_coordinates, y=y_coordinates, z=z_coordinates, mode='markers', marker=markers,
                             text=elements, name='')
        return trace

    def bond_trace():
        """"Creates a bond trace for the plot"""
        trace = go.Scatter3d(x=[], y=[], z=[], hoverinfo='none', mode='lines',
                             marker=dict(color='grey', size=7, opacity=1))
        for i, j in bonds.keys():
            trace['x'] += (x_coordinates[i], x_coordinates[j], None)
            trace['y'] += (y_coordinates[i], y_coordinates[j], None)
            trace['z'] += (z_coordinates[i], z_coordinates[j], None)
        return trace
    
    bonds = get_bonds()
    
    zipped = zip(range(len(elements)), x_coordinates, y_coordinates, z_coordinates)
    annotations_id = [dict(text=num, x=x, y=y, z=z, showarrow=False, yshift=15, font = dict(color = "blue"))
                   for num, x, y, z in zipped]
    
    annotations_length = []
    for (i, j), dist in bonds.items():
        p_i, p_j = Point3D(coordinates[i]), Point3D(coordinates[j])
        p = p_i.midpoint(p_j)
        annotation = dict(text=dist, x=float(p.x), y=float(p.y), z=float(p.z), showarrow=False, yshift=15)
        annotations_length.append(annotation)   
    
    updatemenus = list([
        dict(buttons=list([
                 dict(label = 'Atom indices',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id}]),
                 dict(label = 'Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_length}]),
                 dict(label = 'Atom indices & Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id + annotations_length}]),
                 dict(label = 'Hide all',
                      method = 'relayout',
                      args = [{'scene.annotations': []}])
                 ]),
                 direction='down',
                 xanchor = 'left',
                 yanchor = 'top'
            ),        
    ])
    
    data = [atom_trace(), bond_trace()]
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False, titlefont=dict(color='white'))
    layout = dict(scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params, annotations=annotations_id), 
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=False, updatemenus=updatemenus)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [34]:
df_train = pd.read_csv('./input/train.csv', index_col=0)
df_test = pd.read_csv('./input/test.csv', index_col=0)

In [36]:
df = pd.concat([df_train, df_test], sort=False)

In [39]:
df.drop('scalar_coupling_constant', axis=1, inplace=True)

In [62]:
df.head(10)

,molecule_name,atom_index_0,atom_index_1,type
id,,,,
0,dsgdb9nsd_000001,1,0,1JHC
1,dsgdb9nsd_000001,1,2,2JHH
2,dsgdb9nsd_000001,1,3,2JHH
3,dsgdb9nsd_000001,1,4,2JHH
4,dsgdb9nsd_000001,2,0,1JHC
5,dsgdb9nsd_000001,2,3,2JHH
6,dsgdb9nsd_000001,2,4,2JHH
7,dsgdb9nsd_000001,3,0,1JHC
8,dsgdb9nsd_000001,3,4,2JHH


In [58]:
from collections import defaultdict
types_dict = defaultdict(dict)

In [59]:
g = df.groupby(['molecule_name', 'type'])

In [60]:
for (molecule, t), data in g:
    types_dict[molecule][t] = data[['atom_index_0', 'atom_index_1']].values.tolist()

In [63]:
types_dict['dsgdb9nsd_000002']

{'1JHN': [[1, 0], [2, 0], [3, 0]], '2JHH': [[1, 2], [1, 3], [2, 3]]}

In [64]:
import pickle
with open('./result/types_dict.pkl', 'wb') as f:
    pickle.dump(types_dict, f)

In [66]:
types_dict['dsgdb9nsd_000201'
          ]

{'1JHC': [[6, 0], [7, 0], [8, 0], [9, 2], [11, 4], [12, 5]],
 '1JHN': [[10, 3]],
 '2JHC': [[6, 1],
  [7, 1],
  [8, 1],
  [9, 1],
  [10, 2],
  [10, 4],
  [11, 5],
  [12, 1],
  [12, 4]],
 '2JHH': [[6, 7], [6, 8], [7, 8]],
 '2JHN': [[9, 3], [11, 3]],
 '3JHC': [[6, 2],
  [6, 5],
  [7, 2],
  [7, 5],
  [8, 2],
  [8, 5],
  [9, 0],
  [9, 4],
  [9, 5],
  [10, 1],
  [10, 5],
  [11, 1],
  [11, 2],
  [12, 0],
  [12, 2]],
 '3JHH': [[9, 10], [10, 11], [11, 12]],
 '3JHN': [[12, 3]]}

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict

structures = pd.read_csv('./input/structures.csv')
atomic_radii = dict(C=0.77, F=0.71, H=0.38, N=0.75, O=0.73)
structures['radii'] = structures['atom'].map(atomic_radii)
structures_group = structures.groupby('molecule_name')
molecule_names = structures['molecule_name'].unique()
class MoleculeProperty:
    def __init__(self, molecule_name):
        self.molecule = structures_group.get_group(molecule_name)
        self.coordinates = self.molecule[['x', 'y', 'z']].values
        # self.x_coordinates = self.coordinates[:, 0]
        # self.y_coordinates = self.coordinates[:, 1]
        # self.z_coordinates = self.coordinates[:, 2]
        self.atoms = self.molecule['atom'].tolist()
        self.num = len(self.atoms)
        self.radii = self.molecule['radii'].values
        # atom type
        self.atoms_dict = defaultdict(list)
        for i, atom in enumerate(self.atoms):
            self.atoms_dict[atom].append(i)
        # calculate distance matrix and bulid graph
        self.dist_matrix = self.calc_distance()
        self.bond_distance = self.radii[:, None] + self.radii
        self.graph = nx.Graph(self.dist_matrix < 1.3*self.bond_distance)


    def calc_distance(self):
        d = self.coordinates[:, None, :] - self.coordinates
        return np.sqrt(np.einsum('ijk,ijk->ij', d, d))

    def get_hydrogen_bonds(self):
        hydrogen_bonds = defaultdict(list)
        for start in self.atoms_dict['H'][:]:
            queue = [start]
            visited = {start}
            for i in range(3):
                tmp = []
                while queue:
                    for x1 in self.graph[queue.pop()]:
                        if x1 not in visited:
                            if self.atoms[x1] in {'C', 'N'} or (self.atoms[x1] == 'H' and x1 > start):
                                hydrogen_bonds[f'{i+1}JH{self.atoms[x1]}'].append([start, x1])
                            tmp.append(x1)
                            visited.add(x1)
                queue = tmp
        return hydrogen_bonds



In [59]:
bonds_dict = {}
from multiprocessing import Pool
t0 = time.time()
pool = Pool()
def worker(molecule):
    return (molecule, MoleculeProperty(molecule).get_hydrogen_bonds())
tmp = pool.map(worker, molecule_names)
print(time.time()-t0)

43.859124183654785


In [50]:
bonds_dict = {}
t0 = time.time()
def worker(molecule):
    bonds_dict[molecule] = MoleculeProperty(molecule).get_hydrogen_bonds()
for x in molecule_names:
    worker(x)
print(time.time()-t0)

221.29566383361816


In [2]:
bonds_dict

{'dsgdb9nsd_008457': defaultdict(list,
             {'1JHC': [[8, 0],
               [9, 0],
               [10, 0],
               [11, 1],
               [12, 2],
               [13, 2],
               [14, 4],
               [15, 4],
               [16, 4],
               [17, 5],
               [18, 5],
               [19, 6]],
              '2JHC': [[8, 1],
               [9, 1],
               [10, 1],
               [11, 0],
               [11, 2],
               [11, 6],
               [12, 1],
               [12, 3],
               [13, 1],
               [13, 3],
               [14, 3],
               [15, 3],
               [16, 3],
               [17, 3],
               [17, 6],
               [18, 3],
               [18, 6],
               [19, 1],
               [19, 3],
               [19, 5]],
              '2JHH': [[8, 9],
               [8, 10],
               [9, 10],
               [12, 13],
               [14, 15],
               [14, 16],
               [15, 16],


In [3]:
dict2 = pickle.load(open('./result/types_dict.pkl', 'rb'))

In [ ]:
res = []
for molecule in molecule_names:
    if bonds_dict[molecule] != dict2[molecule]:
        res.append(molecule)
res

In [33]:
bonds_dict[x]['3JHH']

[[3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [4, 5],
 [4, 6],
 [4, 7],
 [4, 8],
 [5, 7],
 [5, 8],
 [6, 7],
 [6, 8]]

In [51]:
# from multiprocessing.dummy import Pool
# pool = Pool()
import time
t0 = time.time()
for k1 in bonds_dict:
    for k2 in bonds_dict[k1]:
        bonds_dict[k1][k2].sort()
print(time.time()-t0)

1.4009621143341064


In [49]:
t0 = time.time()
pool = Pool()
def worker(k1):
    for k2 in bonds_dict1[k1]:
        bonds_dict1[k1][k2].sort()
pool.map(worker, bonds_dict.keys())
print(time.time()-t0)

6.38255500793457


In [42]:
len(bonds_dict)

130775